In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import initializers
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#### Reading the data, normalizing and flattening it

##### Reading tand Normalizing

In [90]:
train = image_dataset_from_directory(
    'Group_24/train/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

val = image_dataset_from_directory(
    'Group_24/val/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

test = image_dataset_from_directory(
    'Group_24/test/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

train = train.map(normalize)
val = val.map(normalize)
test = test.map(normalize)

Found 11385 files belonging to 5 classes.
Found 3795 files belonging to 5 classes.
Found 3795 files belonging to 5 classes.


##### Flattening

Preparing training tensors

In [91]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in train:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
train_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

Preparing validation tensors

In [92]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in val:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
val_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

Preparing testing tensors

In [93]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in test:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
test_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

#### PCA

Reduce the dimensions to 32, 64, 128 and 256 dimensions through PCA

In [94]:
mean = tf.reduce_mean(image_vectors, axis=0)

In [95]:
train_vectors[0] -= mean
val_vectors[0] -= mean
test_vectors[0] -= mean

In [96]:
num_components = [32,64,128,256]

def get_reduced_representation(dim, vec):
    # Calculate the covariance matrix
    cov = tf.matmul(vec[0], vec[0], transpose_a=True)# / tf.cast(tf.shape(vec[0])[0], tf.float32)

    # Perform eigenvalue decomposition on the covariance matrix
    eigenvalues, eigenvectors = tf.linalg.eigh(cov)

    # Sort the eigenvectors by eigenvalues in descending order
    sorted_idx = tf.argsort(eigenvalues, direction='DESCENDING')
    eigenvectors = tf.gather(eigenvectors, sorted_idx, axis=1)

    # Keep only the top k eigenvectors
    top_k_eigenvectors = tf.slice(eigenvectors, [0, 0], [-1, num_components[dim]])

    # Project the data onto the new basis
    reduced_rep = tf.matmul(vec[0], top_k_eigenvectors)
    return reduced_rep

#### Preparing the model

In [97]:
# num_hidden = 3
model_arch = [
    # [4, 4, 4],
    # [4, 8, 16],
    # [16, 8, 4],
    # [8, 8, 8],
    [4, 16, 32],
    [32, 16, 4],
    [16, 16, 16],
    [16, 32, 64],
    [64, 32, 16],
    [32, 32, 32],
    [32, 64, 96],
    [96, 64, 32],
    [64, 64, 64],
    [64, 96, 128],
    [128, 96, 64],
    [128, 128, 128],
    [256, 128, 256],
    [128, 256, 128],
    [256, 256, 256]
]

In [ ]:
initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=42)

k=5 # no. of classes
input_shape = (28, 28, 1)

model_history = dict()

# train different achitectures and optimizers
print('Training models with different architectures and optimizers')
for layer_dims in [model_arch[1]]:
    for reduced_dimension in [32,64,128,256]:
        print(f'{reduced_dimension}-{layer_dims[0]}-{layer_dims[1]}-{layer_dims[2]}...')
        # define model
        model = Sequential([
            keras.Input(shape=(28, 28, 1)),
            layers.Flatten(),
            layers.Dense(layer_dims[0], activation="sigmoid", name="layer1", 
                         kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
            layers.Dense(layer_dims[1], activation="sigmoid", name="layer2", 
                         kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
            layers.Dense(layer_dims[2], activation="sigmoid", name="layer3", 
                         kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
            layers.Dense(k, activation="softmax", name="output", 
                         kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
        ])
        
        # compile model
        model.compile(optimizer=optimizer[1], loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        # callbacks
        my_callbacks = [
            EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=3),
            TensorBoard(log_dir=f'./logdir/{optimizer[0]}-{layer_dims[0]}-{layer_dims[1]}-{layer_dims[2]}/')
        ]
        
        batch_size=1
        if optimizer[0]=='batch':
            batch_size = train.cardinality().numpy()
        
        model_fit = model.fit(train, batch_size=batch_size, epochs=10000, verbose=0, callbacks=my_callbacks, 
                              validation_split=0.0, validation_data=val, shuffle=True, validation_batch_size=None)
        
        model_history[f'{optimizer[0]}-{layer_dims[0]}-{layer_dims[1]}-{layer_dims[2]}'] = model_fit.history['accuracy']
        
        hist_metric = 'accuracy'
        print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
        model.save(f'models/{optimizer[0]}-{layer_dims[0]}-{layer_dims[1]}-{layer_dims[2]}.tf')

